In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#LOAD YOLO
net=cv2.dnn.readNet("C:\\Users\\sanke\\Desktop\\yolo_object_detection\\yolov3.weights","C:\\Users\\sanke\\Desktop\\yolo_object_detection\\yolov3.cfg")
classes=[]
with open("C:\\Users\\sanke\\Desktop\\yolo_object_detection\\coco.names","r") as f:
    classes=[line.strip() for line in f.readlines()]
print(classes)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [3]:
classes1=['car','truck','bus','motorbike']
layer_names=net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))


In [4]:
def classify(path):
    font = cv2.FONT_HERSHEY_PLAIN
    frame = cv2.imread(path)
    height, width, channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True)
    net.setInput(blob)
    outs=net.forward(output_layers)
    class_ids=[]
    confidences=[]
    boxes=[]
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence >  0.1:
                #object detection
                center_x = int(detection[0]*width)
                center_y = int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)
                #rectangle coordinates
                x=int(center_x-w/2)
                y=int(center_y-h/2)
            
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
                indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0, 0.4)
    #print("no. of objects in"+str(path)+ "= ",len(indexes))
    vehicle_count=len(indexes)
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            if label in classes1:
                color=colors[i]
                cv2.rectangle(frame, (x, y), (x + w, y + h),color, 2)
                cv2.putText(frame, label, (x, y), font, 2,color, 3)
    return frame, int(vehicle_count)

In [5]:
def vehicle(signal1,signal2,signal3,signal4):
    a=[signal1,signal2,signal3,signal4]
    v1=[]
    index=[]
    for i in range(4):
        b=max(a)
        c=a.index(b)
        v1.append(b)
        index.append(c)
        a[c]=0

    d=v1
    if d[-1]==0:
        a=d.count(0)
        d=d[:-a]
    else:
        d=d
    return index,d

In [6]:
def vehicle_count():
    import cv2
    font = cv2.FONT_HERSHEY_PLAIN
    video=cv2.VideoCapture('signal1.mp4')
    video1=cv2.VideoCapture('signal2.mp4')
    video2=cv2.VideoCapture('signal3.mp4')
    video3=cv2.VideoCapture('signal4.avi')

    count=0
    while True:
        count+=1
        ret,frame=video.read()
        path='frame.jpg'
        cv2.imwrite(path,frame)
        frame, signal1=classify(path)
        #print("vehicle at signal 1 is ",signal1)
            #count=0

        # signal2
        ret,frame1=video1.read()
        #if count==200:
           # print("count : ",count)
        path='frame1.jpg'
        cv2.imwrite(path,frame1)
        frame, signal2=classify(path)
        #print("vehicle at signal 2 is ",signal2)

        # signal3
        ret,frame2=video2.read()
        #if count==200:
            #print("count : ",count)
        path='frame2.jpg'
        cv2.imwrite(path,frame2)
        frame, signal3=classify(path)
       # print("vehicle at signal 3 is ",signal3)

        # signal4
        ret,frame3=video3.read()
        #if count==200:
            #print("count : ",count)
        path='frame3.jpg'
        cv2.imwrite(path,frame3)
        frame, signal4=classify(path)
        #print("vehicle at signal 4 is ",signal4)

        index,d=vehicle(signal1,signal2,signal3,signal4)
        
        #if count==200:
        img1 = cv2.resize(frame,(360,240))
        img2 = cv2.resize(frame1,(360,240))
        img3 = cv2.resize(frame2,(360,240))
        img4 = cv2.resize(frame3,(360,240))

        font = cv2.FONT_HERSHEY_PLAIN
        n=["frame","frame1","frame2","frame3"]
        
        
        cv2.putText(img1,str(signal1),(50,50),font,5,(255,0,0),4)
        cv2.putText(img2,str(signal2),(50,50),font,5,(255,0,0),4)
        cv2.putText(img3,str(signal3),(50,50),font,5,(255,0,0),4)
        cv2.putText(img4,str(signal4),(50,50),font,5,(255,0,0),4)
        cv2.imshow("img1",img1)
        cv2.imshow("img2",img2)
        cv2.imshow("img3",img3)
        cv2.imshow("img4",img4)
        if cv2.waitKey(5000) & count==10:
            cv2.destroyAllWindows()
        break

    video.release()
    video1.release()
    video2.release()
    video3.release()
    cv2.destroyAllWindows()
    return index,d    

In [7]:
import turtle
import time
from playsound import playsound

turtle.bgcolor("black")

def loc(x,y):
    n=turtle.Turtle()
    n.shape("square")
    n.shapesize(2,2,4)
    n.color("grey")
    n.penup()
    n.goto(x,y)
def timer(x,y,sec):
    t=turtle.Turtle()
    t.color("red")
    t.hideturtle()
    t.penup()
    t.goto(x,y)

    for i in range(sec):
        t.write(sec-i,font=("Arial", 15, "normal"))
        time.sleep(1)
        t.clear()

        
pen=turtle.Turtle()
pen.color("YELLOW")
pen.width(3)
pen.hideturtle()
pen.penup()
pen.goto(-150,0)
pen.pendown()
pen.fd(60)
pen.rt(90)
pen.fd(120)
pen.rt(90)
pen.fd(60)
pen.rt(90)
pen.fd(120)
pen.write("signal1")
loc(-130,40)

pen=turtle.Turtle()
pen.color("YELLOW")
pen.width(3)
pen.hideturtle()
pen.penup()
pen.goto(150,0)
pen.pendown()
pen.fd(60)
pen.rt(90)
pen.fd(120)
pen.rt(90)
pen.fd(60)
pen.rt(90)
pen.fd(120)
pen.write("signal2")
loc(170,40)

pen=turtle.Turtle()
pen.color("YELLOW")
pen.width(3)
pen.hideturtle()
pen.penup()
pen.goto(0,160)
pen.pendown()
pen.fd(60)
pen.rt(90)
pen.fd(120)
pen.rt(90)
pen.fd(60)
pen.rt(90)
pen.fd(120)
pen.write("signal3")
loc(20,200)

pen=turtle.Turtle()
pen.color("YELLOW")
pen.width(3)
pen.hideturtle()
pen.penup()
pen.goto(0,-160)
pen.pendown()
pen.fd(60)
pen.rt(90)
pen.fd(120)
pen.rt(90)
pen.fd(60)
pen.rt(90)
pen.fd(120)
pen.write("signal4")
loc(20,-120)

# RED light
red_light=turtle.Turtle()
red_light.shape("circle")
red_light.color("grey")
red_light.penup()
red_light.goto(-120,-20) 

red_light1=turtle.Turtle()
red_light1.shape("circle")
red_light1.color("grey")
red_light1.penup()
red_light1.goto(180,-20)

red_light2=turtle.Turtle()
red_light2.shape("circle")
red_light2.color("grey")
red_light2.penup()
red_light2.goto(30,140)

red_light3=turtle.Turtle()
red_light3.shape("circle")
red_light3.color("grey")
red_light3.penup()
red_light3.goto(30,-180)

# YELLOW light
yellow_light=turtle.Turtle()
yellow_light.shape("circle")
yellow_light.color("grey")
yellow_light.penup()
yellow_light.goto(-120,-60)

yellow_light1=turtle.Turtle()
yellow_light1.shape("circle")
yellow_light1.color("grey")
yellow_light1.penup()
yellow_light1.goto(180,-60)

yellow_light2=turtle.Turtle()
yellow_light2.shape("circle")
yellow_light2.color("grey")
yellow_light2.penup()
yellow_light2.goto(30,100)

yellow_light3=turtle.Turtle()
yellow_light3.shape("circle")
yellow_light3.color("grey")
yellow_light3.penup()
yellow_light3.goto(30,-220)

#GREEn light
green_light=turtle.Turtle()
green_light.shape("circle")
green_light.color("grey")
green_light.penup()
green_light.goto(-120,-100)

green_light1=turtle.Turtle()
green_light1.shape("circle")
green_light1.color("grey")
green_light1.penup()
green_light1.goto(180,-100)

green_light2=turtle.Turtle()
green_light2.shape("circle")
green_light2.color("grey")
green_light2.penup()
green_light2.goto(30,60)

green_light3=turtle.Turtle()
green_light3.shape("circle")
green_light3.color("grey")
green_light3.penup()
green_light3.goto(30,-260)
    
def show_light(): 
    red=[red_light,red_light1,red_light2,red_light3]
    green=[green_light,green_light1,green_light2,green_light3]
    yellow=[yellow_light,yellow_light1,yellow_light2,yellow_light3]
    position=[[-130,40],[170,40],[20,200],[20,-120]]

    a1="Sound_1.mp3"
    a2="Sound_2.mp3"
    a3="Sound_3.mp3"
    a4="Sound_4.mp3"
    sound=[a1,a2,a3,a4]
    while True:
        index,d=vehicle_count()
        print(index)
        print(d)
        t=[0,0,0,0]
        for i in range(len(d)):
            if d[i]==0:
                t[i]=0
            elif d[i]<=10:
                t[i]=10
            elif d[i]>=11 and d[i]<30:
                t[i]=25
            elif d[i]>=30 and d[i]<50:
                t[i]=40
            else:
                t[i]=60

        total=t[0]+t[1]+t[2]+t[3]+12
        print(total)
        t1=0
        #while t1<=total:
        if len(d)==4:
            red[index[0]].color('grey')

            # 1st cycle
            if d[0]!=0:
                green[index[0]].color('green')
                playsound(sound[index[0]])
            red[index[1]].color('red')
            red[index[2]].color('red')
            red[index[3]].color('red')
            timer(position[index[0]][0],position[index[0]][1],t[0])
            t1=t1+t[0]
            #time.sleep(10)

            green[index[0]].color('grey')
            yellow[index[0]].color('yellow')
            red[index[1]].color('red')
            red[index[2]].color('red')
            red[index[3]].color('red')
            timer(position[index[0]][0],position[index[0]][1],3)
            t1=t1+3
            #time.sleep(3)

            # 2nd signal start 1st signal end
            yellow[index[0]].color('grey')
            red[index[0]].color('red')
            red[index[1]].color('grey')
            if d[1]!=0:
                green[index[1]].color('green')
                playsound(sound[index[1]])
            red[index[2]].color('red')
            red[index[3]].color('red')
            timer(position[index[1]][0],position[index[1]][1],t[1])
            t1=t1+t[3]
            #time.sleep(10)

            green[index[1]].color('grey')
            yellow[index[1]].color('yellow')
            red[index[0]].color('red')
            red[index[2]].color('red')
            red[index[3]].color('red')
            timer(position[index[1]][0],position[index[1]][1],3)
            t1=t1+3
            #time.sleep(3)

            # 3rd signal start 2nd signal end
            yellow[index[1]].color('grey')
            red[index[1]].color('red')
            red[index[2]].color('grey')
            if d[2]!=0:
                green[index[2]].color('green')
                playsound(sound[index[2]])
            red[index[0]].color('red')
            red[index[3]].color('red')
            timer(position[index[2]][0],position[index[2]][1],t[2])
            t1=t1+t[2]
            #time.sleep(10)

            green[index[2]].color('grey')
            yellow[index[2]].color('yellow')
            red[index[0]].color('red')
            red[index[1]].color('red')
            red[index[3]].color('red')
            timer(position[index[2]][0],position[index[2]][1],3)
            t1=t1+3
            #time.sleep(3)

            # 4th signal start and 3rd signal end
            yellow[index[2]].color('grey')
            red[index[2]].color('red')
            red[index[3]].color('grey')
            if d[-1]!=0:
                green[index[3]].color('green')
                playsound(sound[index[3]])
            red[index[0]].color('red')
            red[index[1]].color('red')
            timer(position[index[3]][0],position[index[3]][1],t[3])
            t1=t1+t[3]
            #time.sleep(10)
            green[index[3]].color('grey')
            yellow[index[3]].color('yellow')
            red[index[0]].color('red')
            red[index[1]].color('red')
            red[index[2]].color('red')    
            t1=t1-3
            timer(position[index[3]][0],position[index[3]][1],3)
            #time.sleep(3)

            yellow[index[3]].color('grey')
            red[index[3]].color('red')
        

show_light()

[2, 0, 3, 1]
[21, 19, 9, 7]
82


Terminator: 